In [ ]:
# default_exp core

In [ ]:
#hide
# from google.colab import drive
# drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
#hide
!pip install fastcore
from fastcore.all import *
%load_ext autoreload 
%autoreload 2
%cd "/content/drive/MyDrive/Coding/ModelAssistedLabel"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/content/drive/MyDrive/Coding/ModelAssistedLabel


In [ ]:
# hide
%run "_Synch.ipynb"

In [ ]:
repo = "/content/drive/MyDrive/Coding/Roboflow Export (841)"
name = "nospaces"
wm = AutoWeights(repo, name)

summary:  [{'train': 4}, {'valid': 1}, {'test': 0}]
checksum: 5


AttributeError: ignored